### 导入 MNIST 数据库

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
# 失效了

In [2]:
import scipy.io
mnist = scipy.io.loadmat('mnist-original.mat')

In [3]:
X, y = mnist['data'], mnist['label'] 
print(X.shape, y.shape) 

# 共七万份数据，每一个数据都是一个 784 位的列向量

(784, 70000) (1, 70000)


In [4]:
X = np.array(X, dtype=float).T
y = np.array(y).T

In [5]:
X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [6]:
X_train.shape

(60000, 784)

### 使用 KNN 进行手写体识别

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
%time knn_clf.fit(X_train, y_train)

Wall time: 50 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [8]:
%time knn_clf.score(X_test, y_test)

Wall time: 12.5 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9688

这里不需要进行数据归一化，因为对于每一个像素点，其特征值（即亮度）的范围是一样的。

### PCA 数据降维

In [19]:
from sklearn.decomposition import PCA

pca = PCA(0.9) 
pca.fit(X_train) # 训练 PCA 模型

PCA(n_components=0.9)

In [20]:
X_train_reduction = pca.transform(X_train)
X_test_reduction  = pca.transform(X_test)
X_train_reduction.shape # 从700多维度的数据降至87维

(60000, 87)

In [21]:
knn_clf = KNeighborsClassifier()
%time knn_clf.fit(X_train_reduction, y_train)

Wall time: 9 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [22]:
%time knn_clf.score(X_test_reduction, y_test)
# 感觉也没有快多少））

Wall time: 8.5 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9728

识别准确率不降反升，是由于PCA还可以对数据进行降噪处理。